# Train Vision Transformer for a single audio time slice

We make raw code blocks for more user functionality. Convert these to code blocks, thereby making custom models outside the scope of the command line interface.

### Imports

In [1]:
import os
import joblib

# array data manipulation and plotting
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from transformers import ViTConfig, ViTForImageClassification, ViTFeatureExtractor

# machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, \
    accuracy_score, \
    f1_score, \
    auc, \
    recall_score, \
    precision_score, \
    precision_recall_curve, \
    roc_curve
from sklearn.metrics import confusion_matrix

2025-11-13 21:06:57.800583: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 21:06:57.836103: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# Example for single-channel image
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # Resize to pretrained ViT size
    transforms.ToTensor(),          # Converts to [0,1]
])

In [4]:
batch_size = 2**4
X_train_path = os.path.join('data', 'X_train.npy')
X_test_path = os.path.join('data', 'X_test.npy')
Y_train_path = os.path.join('data', 'Y_train.npy')
Y_test_path = os.path.join('data', 'Y_test.npy')

In [5]:
# --- LOAD DATA ---
X_train = np.load(X_train_path)
X_test = np.load(X_test_path)
Y_train = np.load(Y_train_path)
Y_test = np.load(Y_test_path)
num_classes = len(np.unique(Y_train))

# --- MAKE TENSORS ---
Y_train = torch.tensor(Y_train, dtype=torch.long)
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)

# Apply to a single image (H, W) or (C, H, W)
X_test = torch.stack([transform(img) for img in X_test])  # batch dimension
X_train = torch.stack([transform(img) for img in X_train])  # batch dimension

# --- DATA LOADER ---
train_data = TensorDataset(X_train, Y_train)
test_data = TensorDataset(X_test, Y_test)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# X_height = X_train.shape[1]
# X_width = X_train.shape[2]

del X_train, Y_train, X_test, Y_test

### Set up model

In [6]:
def repeat_channels(x):
    # x: [batch, 1, H, W]
    return x.repeat(1, 3, 1, 1)

In [9]:
# Load pretrained ViT
model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name,
                                                 )
# Feature extractor (normalizes images the way pretrained ViT expects)
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)

model.config.num_labels = num_classes
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)

# only train the classifier component
for param in model.vit.parameters():
    param.requires_grad = False

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

In [ ]:

# criterion choice
criterion = torch.nn.CrossEntropyLoss()

# choice of optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.001)

model

### Train the model

In [ ]:
# Training loop

model.to(device)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Pass to GPU
        inputs = repeat_channels(inputs)
        inputs = feature_extractor(inputs, return_tensors='pt')
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(**inputs)

        # Compute the loss
        loss = criterion(outputs.logits, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        running_loss += loss.item()

    # Print the average loss for this epoch
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/10, Loss: 2.2829


In [49]:
# Evaluate the model's accuracy on the validation data
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation for evaluation
    for inputs, labels in train_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get the predicted class (index of the maximum value in the output)
        # For multi-class (one-hot) targets
        predicted = outputs.logits.argmax(dim=1)
        labels = labels.int()
        predicted = predicted.cpu()
        labels = labels.cpu()
        print(accuracy_score(predicted,labels))

0.21484375
0.16796875
0.19140625
0.18359375
0.2265625
0.21484375
0.24609375
0.21484375
0.2578125
0.19140625
0.16796875
0.171875
0.18359375
0.1953125
0.1875
0.19921875
0.203125
0.24609375
0.16796875
0.21875
0.16015625
0.21484375
0.18359375
0.234375
0.15625
0.1640625
0.2109375
0.20703125
0.1796875
0.21875
0.23828125
0.2109375
0.20703125
0.1953125
0.234375
0.1484375
0.22265625
0.2265625
0.22265625
0.13671875
0.1640625
0.1484375
0.171875
0.1640625
0.1796875
0.18359375
0.21875
0.17578125
0.20703125
0.234375
0.21484375
0.19140625
0.16796875
0.1640625
0.16796875
0.1953125
0.2109375
0.17578125
0.15625
0.203125
0.16796875
0.1953125
0.15625
0.1953125
0.14453125
0.16796875
0.2109375
0.1875
0.1796875
0.234375
0.17578125
0.2265625
0.18359375
0.171875
0.23828125
0.18359375
0.20703125
0.1640625
0.1875
0.1796875
0.265625
0.15625
0.20703125
0.23046875
0.19921875
0.25
0.16796875
0.1875
0.171875
0.2109375
0.21484375
0.23046875
0.2109375
0.18359375
0.2109375
0.1640625
0.18359375
0.1640625
0.1640625
0.1796

In [50]:
# Evaluate the model's accuracy on the validation data
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient computation for evaluation
    for inputs, labels in test_loader:

        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get the predicted class (index of the maximum value in the output)
        # For multi-class (one-hot) targets
        predicted = outputs.logits.argmax(dim=1)
        labels = labels.int()
        predicted = predicted.cpu()
        labels = labels.cpu()
        print(accuracy_score(predicted,labels))

0.05859375
0.03515625
0.0
0.58984375
0.484375
0.453125
0.22265625
0.0
0.0
0.1015625
0.08984375
0.03515625
0.0
0.0
